# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 6.6 MB 36.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
import torch
DEVICE = torch.device("cuda:0")

In [3]:
# загружаем модель
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [4]:
from google.colab import drive

ROOT_DIR = '/content/drive' 
drive.mount(ROOT_DIR)

# обучающие данные
train_path = 'drive/MyDrive/CL_hw/great_Satan_ball.txt'

Mounted at /content/drive


In [5]:
# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
  
# специальный класс, который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [6]:
from transformers import Trainer, TrainingArguments

In [7]:
training_args = TrainingArguments( 
    output_dir= 'drive/MyDrive/CL_hw/finetuned',
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [8]:
trainer.train()

***** Running training *****
  Num examples = 116
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.6773526000976563, metrics={'train_runtime': 465.0033, 'train_samples_per_second': 24.946, 'train_steps_per_second': 0.215, 'total_flos': 378873446400000.0, 'train_loss': 0.6773526000976563, 'epoch': 100.0})

In [17]:
text_1 = '– Где же гости? – спросила Маргарита.'
text_2 = 'Тогда произошла метаморфоза.'
input_ids_1 = tokenizer.encode(text_1, return_tensors='pt').to(DEVICE)
input_ids_2 = tokenizer.encode(text_2, return_tensors='pt').to(DEVICE)

## Argmax

In [18]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– Я приглашаю вас на бал, – ответил ей Коровьев, – а вы, королева, будете королевой бала.

Маргарита, не в силах больше стоять, упала на

Sample 2
Тогда произошла метаморфоза.

– А, – сказал Коровьев, – я рад, что вы так быстро пришли в себя, Маргарита. Я рад, что вы пришли ко мне в гости.

– Я рада, –


## Beam Search

#### num_beams = 5

In [19]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        num_beams=5,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        num_beams=5,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– Я приглашен, – отвечал Коровьев.

– А где же Маргарита? – спросила Маргарита.

– Я приглашен, – отвечал Коровьев.

–

Sample 2
Тогда произошла метаморфоза.

– Я в восхищении, – сказал Коровьев.

– Я в восхищении, – сказал Воланд.

– Я в восхищении, – сказал Коровьев.




#### num_beams = 10

In [20]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1, 
                        do_sample=False,
                        num_beams=10,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=False,
                        num_beams=10,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– Мы в восхищении, сударыня, – ответил Коровьев.

– А я в восхищении, – сказал Коровьев.

– Я в восхищении, –

Sample 2
Тогда произошла метаморфоза. У Маргариты на глазах выступили слезы.

– Я в восхищении, королева, – шептал Коровьев.

– Я в восхищении, королева, – шептал Коровьев.

–


## Сэмплирование с Температурой

#### temperature = 0.05

In [21]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=0.05,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=0.05,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– Я, королева, – ответил Коровьев, – приглашаю вас в свой дом.

– Я, – сказала Маргарита, – рада вас видеть.

– Я, –

Sample 2
Тогда произошла метаморфоза. Маргарите показалось, что она видит перед собой не просто женщину, а женщину с женщиной.

– Я рада, – сказала Маргарита, – что вы пришли ко мне в гости. Я рада, что вы пришли


#### temperature = 0.75

In [22]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=0.75,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=0.75,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– Загорели, – отвечал ей Коровьев, – но вчера их было много, как много, и они все не расходились.

– А где хозяин? – спросила Маргарита

Sample 2
Тогда произошла метаморфоза.

– Да, – прошептала Маргарита, – все было в полном порядке. И еще один человек, которого вы, наверное, не знаете, был моим другом. Я счастлива познакомиться с ним в этой компании.



#### temperature = 1, top_k = 20

In [23]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=1,
                        top_k=20,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=1,
                        top_k=20,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– Мы сегодня с вами, господа хорошие, в удивлении, госпожа Королева, – ответил за всех, высокий и дородный мужчина, – познакомились со своими гостями, и мы хотели бы задать

Sample 2
Тогда произошла метаморфоза. Теперь ее видели как бы вживую, на одной левой руке, с поднятым локтем, а все тело с отвислым животом. На голове Маргариты – черный колпак, с которого капала


#### temperature = 0.75, top_k = 20

In [24]:
with torch.no_grad():
    out_1 = model.generate(input_ids_1,
                        do_sample=True,
                        temperature=0.75,
                        top_k=10,
                        max_length=50,
                        )
    out_2 = model.generate(input_ids_2, 
                        do_sample=True,
                        temperature=0.75,
                        top_k=10,
                        max_length=50,
                        )

generated_text_1 = list(map(tokenizer.decode, out_1))[0]
generated_text_2 = list(map(tokenizer.decode, out_2))[0]
print()
print('Sample 1')
print(generated_text_1)
print()
print('Sample 2')
print(generated_text_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Sample 1
– Где же гости? – спросила Маргарита.

– В зале, – ответил Коровьев, – но они не будут играть, королева.

– Королева! – закричал Воланд.

– Королева? – переспросил Воланд.

Sample 2
Тогда произошла метаморфоза.

– Как? – в ужасе спросила Маргарита.

– Ушиблась и сломана правая рука, – пояснил Коровьев, – и теперь она не может ходить.

– Я в восхи


### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

__Ответ:__ GPT-1 был представлени в статье __Improving Language Understanding by Generative Pre-Training__ (Radford, A., & Narasimhan, K. (2018));

GPT-2 в __Language Models are Unsupervised Multitask Learners Radford__ (A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019)(; 

GPT-3 в __Language Models are Few-Shot Learners__ (Brown, Tom B., Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared Kaplan, Prafulla Dhariwal, Arvind Neelakantan, Pranav Shyam, Girish Sastry, Amanda Askell, Sandhini Agarwal, Ariel Herbert-Voss, Gretchen Krueger, T. J. Henighan, Rewon Child, Aditya Ramesh, Daniel M. Ziegler, Jeff Wu, Clemens Winter, Christopher Hesse, Mark Chen, Eric Sigler, Mateusz Litwin, Scott Gray, Benjamin Chess, Jack Clark, Christopher Berner, Sam McCandlish, Alec Radford, Ilya Sutskever and Dario Amodei).

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

__Ответ:__ 60% обучающей выборки состояло из отфильтрованного корпуса Common Crawl (около триллиона слов) с данными за 2016-2019 годы, кроме него использовались датасеты WebText2, Books1, Books2 и Wikipedia. 

Создатели старались обеспечить высокое качество с помощью автоматической фильтрации корпуса, посл которой из него исчезли тексты низкого качества; они провели дедупликацию на уровне документа для предотвращения избыточности; добавили датасеты с текстами высокого качества (WebText2, Books1, Books2 и Wikipedia).